# Destilación fermiónica (ejemplo)

Primero inicializamos el programa. Vamos a trabajar en dimensión 6.

In [3]:
using Fermionic

o = Op(6);
vac = vacuum(o);

cd1 = cdm(o,1);
cd2 = cdm(o,2);
cd3 = cdm(o,3);
cd4 = cdm(o,4);
cd5 = cdm(o,5);
cd6 = cdm(o,6);

**0.** El estado inicial es

> $|\psi_0\rangle = (\sqrt{0.9} c_1^\dagger c_3^\dagger + \sqrt{0.1} c_2^\dagger c_4^\dagger)|0\rangle $

In [24]:
p1 = sqrt(0.9)
p2 = sqrt(1-p1^2)

psi0 = (p1*cd1*cd3+p2*cd2*cd4)*vac
psi0 = State_sparse(psi0,o) #Inicializo el estado (así puedo acceder a su matriz de un cuerpo)
println("Los autovalores de la matriz de un cuerpo son: ",eigensp(psi0))
println("La entropía de un cuerpo es: ",ssp(psi0))

Los autovalores de la matriz de un cuerpo son: [0.0, 0.0, 0.1, 0.1, 0.9, 0.9]
La entropía de un cuerpo es: 0.3126637290595208


**1.** Le agregamos una ancilla. Esta ancilla va a ser un fermión en un subsistema de dimensión 2 de la siguiente forma

> $|\chi_{\rm ancilla}\rangle = \frac{1}{\sqrt{2}}(c_5^\dagger + c_6^\dagger)|0\rangle$

 Nuestro estado queda ahora como

> $|\psi_1\rangle = |\chi_{\rm ancilla}\rangle |\psi_0\rangle  = \frac{1}{\sqrt{2}}(c_5^\dagger + c_6^\dagger) (\sqrt{0.9} c_1^\dagger c_3^\dagger + \sqrt{0.1} c_2^\dagger c_4^\dagger)|0\rangle$

In [25]:
a = 1/sqrt(2)
b = sqrt(1-a^2);

psi1 = (a*cd5 + b*cd6)*(p1*cd1*cd3+p2*cd2*cd4)*vac

64-element SparseArrays.SparseVector{Float64,Int64} with 4 stored entries:
  [22]  =  0.223607
  [23]  =  0.223607
  [42]  =  0.67082
  [43]  =  0.67082

In [39]:
psi1_s = State_sparse(psi1, o)
print("los autovalores siguen siendo ", eigensp(psi1_s), " y la entropía sigue siendo ", ssp(psi1_s))

los autovalores siguen siendo [0.0, 0.1, 0.1, 0.9, 0.9, 1.0] y la entropía sigue siendo 0.3126637290595208

La entropia de un cuerpo se mantiene constante. Pareciera ser una operación libre en este sentido.

Veamos los elementos de la base que componen este estado

In [27]:
println(Array(basis(o)[22,:]))
println(Array(basis(o)[23,:]))
println(Array(basis(o)[42,:]))
println(Array(basis(o)[43,:]))

[0.0, 1.0, 0.0, 1.0, 0.0, 1.0]
[0.0, 1.0, 0.0, 1.0, 1.0, 0.0]
[1.0, 0.0, 1.0, 0.0, 0.0, 1.0]
[1.0, 0.0, 1.0, 0.0, 1.0, 0.0]


**2.** Aplicamos una operación control not, con el modo 5 (ancilla) como control, y los modos 1 y 2 como targets
> $|\psi_2\rangle = U_{\rm CNOT}^{5: 1,2}|\psi_1\rangle = U_{\rm CNOT}^{5: 1,2} \frac{1}{\sqrt{2}}(c_5^\dagger + c_6^\dagger) (\sqrt{0.9} c_1^\dagger c_3^\dagger + \sqrt{0.1} c_2^\dagger c_4^\dagger)|0\rangle
\\= \frac{1}{\sqrt{2}}[c_5^\dagger (\sqrt{0.9} c_2^\dagger c_3^\dagger + \sqrt{0.1} c_1^\dagger c_4^\dagger)+ c_6^\dagger(\sqrt{0.9} c_1^\dagger c_3^\dagger + \sqrt{0.1} c_2^\dagger c_4^\dagger)] |0\rangle$

Esta operación no es libre en nuestra teoría, pero siento que debería serlo para tener una teoría de recursos aplicable.

In [28]:
psi2 = ucnot(o, 5, 1)*ucnot(o, 5, 2)*psi1

64-element SparseArrays.SparseVector{Float64,Int64} with 4 stored entries:
  [22]  =  0.223607
  [27]  =  0.67082
  [39]  =  0.223607
  [42]  =  0.67082

**3.** Usamos una Hadamard en los modos de la ancilla 5 y 6

> $|\psi_3\rangle = H|\psi_2\rangle = H \frac{1}{\sqrt{2}}[c_5^\dagger (\sqrt{0.9} c_2^\dagger c_3^\dagger + \sqrt{0.1} c_1^\dagger c_4^\dagger)+ c_6^\dagger(\sqrt{0.9} c_1^\dagger c_3^\dagger + \sqrt{0.1} c_2^\dagger c_4^\dagger)] |0\rangle\\
= \frac{1}{2}[(c_5^\dagger + c_6^\dagger)(\sqrt{0.9} c_2^\dagger c_3^\dagger + \sqrt{0.1} c_1^\dagger c_4^\dagger) + (c_5^\dagger - c_6^\dagger)(\sqrt{0.9} c_1^\dagger c_3^\dagger + \sqrt{0.1} c_2^\dagger c_4^\dagger)]|0\rangle$

Esta operación es una unitaria de un cuerpo en dos modos, con lo cual es libre

In [29]:
psi3 = hadamard(o, 5, 6)*psi2

64-element SparseArrays.SparseVector{Float64,Int64} with 8 stored entries:
  [22]  =  0.158114
  [23]  =  0.158114
  [26]  =  0.474342
  [27]  =  0.474342
  [38]  =  0.158114
  [39]  =  0.158114
  [42]  =  0.474342
  [43]  =  0.474342

**4.** Nuevamente aplicamos una transformación control usando nuevamente el mismo modo de la ancilla como control (abajo en una nota, me dí cuenta de que usando el otro modo como control mejoraba todavía más el resultado), pero ahora los modos 3 y 4 como target

> $|\psi_4\rangle = U_{\rm CNOT}^{5: 3,4}|\psi_3\rangle = U_{\rm CNOT}^{5: 3,4} \frac{1}{2}[(c_5^\dagger + c_6^\dagger)(\sqrt{0.9} c_2^\dagger c_3^\dagger + \sqrt{0.1} c_1^\dagger c_4^\dagger) + (c_5^\dagger - c_6^\dagger)(\sqrt{0.9} c_1^\dagger c_3^\dagger + \sqrt{0.1} c_2^\dagger c_4^\dagger)]|0\rangle\\
=U_{\rm CNOT}^{5: 3,4} \frac{1}{2}[c_5^\dagger (\sqrt{0.9} c_2^\dagger c_3^\dagger + \sqrt{0.1} c_1^\dagger c_4^\dagger + \sqrt{0.9} c_1^\dagger c_3^\dagger + \sqrt{0.1} c_2^\dagger c_4^\dagger) +  c_6^\dagger(\sqrt{0.9} c_2^\dagger c_3^\dagger + \sqrt{0.1} c_1^\dagger c_4^\dagger - \sqrt{0.9} c_1^\dagger c_3^\dagger - \sqrt{0.1} c_2^\dagger c_4^\dagger)]|0\rangle\\
=\frac{1}{2}[c_5^\dagger (\sqrt{0.9} c_2^\dagger c_4^\dagger + \sqrt{0.1} c_1^\dagger c_3^\dagger + \sqrt{0.9} c_1^\dagger c_4^\dagger + \sqrt{0.1} c_2^\dagger c_3^\dagger) +  c_6^\dagger(\sqrt{0.9} c_2^\dagger c_3^\dagger + \sqrt{0.1} c_1^\dagger c_4^\dagger - \sqrt{0.9} c_1^\dagger c_3^\dagger - \sqrt{0.1} c_2^\dagger c_4^\dagger)]|0\rangle$

In [30]:
psi4 = ucnot(o, 5, 3)*ucnot(o, 5, 4)*psi3

64-element SparseArrays.SparseVector{Float64,Int64} with 8 stored entries:
  [22]  =  0.158114
  [23]  =  0.474342
  [26]  =  0.474342
  [27]  =  0.158114
  [38]  =  0.158114
  [39]  =  0.474342
  [42]  =  0.474342
  [43]  =  0.158114

**5.** Vamos ahora a proyectar sobre la ocupación de los modos 1 o 2 y quedarnos con el sistema compuesto por los modos 3,4,5 y 6
> $|\psi_4\rangle = \frac{1}{2}(\sqrt{0.9} c_5^\dagger c_2^\dagger c_4^\dagger  + \sqrt{0.1} c_5^\dagger c_1^\dagger c_3^\dagger + \sqrt{0.9} c_5^\dagger c_1^\dagger c_4^\dagger + \sqrt{0.1} c_5^\dagger c_2^\dagger c_3^\dagger  +  \sqrt{0.9} c_6^\dagger c_2^\dagger c_3^\dagger + \sqrt{0.1} c_6^\dagger c_1^\dagger c_4^\dagger - \sqrt{0.9} c_6^\dagger c_1^\dagger c_3^\dagger - \sqrt{0.1}c_6^\dagger c_2^\dagger c_4^\dagger)|0\rangle\\
= -\frac{1}{2}(\sqrt{0.9}  c_2^\dagger c_5^\dagger c_4^\dagger  + \sqrt{0.1}  c_1^\dagger c_5^\dagger c_3^\dagger + \sqrt{0.9}  c_1^\dagger c_5^\dagger c_4^\dagger + \sqrt{0.1}  c_2^\dagger c_5^\dagger c_3^\dagger  +  \sqrt{0.9}  c_2^\dagger c_6^\dagger c_3^\dagger + \sqrt{0.1}  c_1^\dagger c_6^\dagger c_4^\dagger - \sqrt{0.9}  c_1^\dagger c_6^\dagger c_3^\dagger - \sqrt{0.1} c_2^\dagger c_6^\dagger c_4^\dagger)|0\rangle\\
= -\frac{1}{2}[c_1^\dagger(\sqrt{0.1}   c_5^\dagger c_3^\dagger + \sqrt{0.9}  c_5^\dagger c_4^\dagger + \sqrt{0.1}   c_6^\dagger c_4^\dagger - \sqrt{0.9}  c_6^\dagger c_3^\dagger) + c_2^\dagger(\sqrt{0.9} c_5^\dagger c_4^\dagger  +   \sqrt{0.1}   c_5^\dagger c_3^\dagger  +  \sqrt{0.9}   c_6^\dagger c_3^\dagger   - \sqrt{0.1} c_6^\dagger c_4^\dagger)]|0\rangle$

> $|\psi_5\rangle = P_1 |\psi_4\rangle = N(\sqrt{0.1}   c_5^\dagger c_3^\dagger + \sqrt{0.9}  c_5^\dagger c_4^\dagger + \sqrt{0.1}   c_6^\dagger c_4^\dagger - \sqrt{0.9}  c_6^\dagger c_3^\dagger)|0\rangle$

con N la normalización. En realidad estoy midiendo la ocupación en los modos 1-2, no importa el resultado que me de. En cuaqluiera de los dos casos, el estado final de 3,4,5,6 tiene los mismos autovalores.

In [40]:
psi5 = cm(o,1)*psi4

64-element SparseArrays.SparseVector{Float64,Int64} with 4 stored entries:
  [6 ]  =  0.158114
  [7 ]  =  0.474342
  [10]  =  0.474342
  [11]  =  0.158114

In [41]:
psi5 = psi5/sqrt(psi5'*psi5)

64-element SparseArrays.SparseVector{Float64,Int64} with 4 stored entries:
  [6 ]  =  0.223607
  [7 ]  =  0.67082
  [10]  =  0.67082
  [11]  =  0.223607

In [42]:
final = State_sparse(psi5,o);

In [45]:
print("los autovalores son ahora ", eigensp(final), " y la entropía aumentó de ",ssp(psi1_s), " a ",ssp(final))

los autovalores son ahora [0.0, 0.0, 0.2, 0.2, 0.8, 0.8] y la entropía aumentó de 0.3126637290595208 a 0.4812853965915749

Si proyectaba al modo 2, llegaba exactamente a los mismos autovalores.

In [46]:
psi6 = cm(o,2)*psi4
psi6 = psi6/sqrt(psi6'*psi6)
final2 = State_sparse(psi6,o);
eigensp(final2)

6-element Array{Float64,1}:
 0.0
 0.0
 0.2
 0.2
 0.8
 0.8

# Nota: mejora en el protocolo
 Me dí cuenta despues de que puede mejorarse todavía más el entrelazamiento conseguido. Si cambio el control en el paso 4 para que controle el modo 6:

In [49]:
psi4_2 = ucnot(o, 6, 3)*ucnot(o, 5, 4)*psi3
psi5_2 = cm(o,1)*psi4_2
psi5_2 = psi5_2/sqrt(psi5_2'*psi5_2)
final_2 = State_sparse(psi5_2,o)

print("los autovalores son ahora ", eigensp(final_2), " y la entropía aumentó de ",ssp(psi1_s), " a ",ssp(final_2))

los autovalores son ahora [0.0, 0.0, 0.2, 0.5, 0.5, 0.8] y la entropía aumentó de 0.3126637290595208 a 0.5739760316291207